In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium 
from shapely.geometry import Point, Polygon 

In [ ]:
nyc_squirrels=pd.read_csv('2018_Central_Park_Squirrel_Census_-_Squirrel_Data_20250304.csv')

In [ ]:
hectare = pd.read_csv('2018_Central_Park_Squirrel_Census_-_Hectare_Grid_20250304.csv')


In [ ]:
#seperating the latitude and longitude 
import pandas as pd
import re


# Extract coordinate pairs using regex
def extract_coords(wkt):
    # Remove the leading "MULTIPOLYGON (((" and trailing ")))"
    cleaned = re.sub(r'^MULTIPOLYGON\s+\(\(\(|\)\)\)$', '', wkt)
    # Split into individual coordinate pairs
    coord_pairs = cleaned.split(', ')
    # Split each pair into lon and lat
    coords = [coord.split() for coord in coord_pairs]
    return coords

# Apply extraction
hectare['coords'] = hectare['the_geom'].apply(extract_coords)

# Convert coordinate lists to individual columns
# Flatten coordinates into new DataFrame columns
coord_df = hectare['coords'].apply(lambda x: pd.Series([float(num) for pair in x for num in pair]))

# Rename the columns as lon_1, lat_1, lon_2, lat_2, ...
coord_df.columns = [f"{'lon' if i % 2 == 0 else 'lat'}_{i//2 + 1}" for i in coord_df.columns]

# Concatenate with original DataFrame
hectare_final = pd.concat([hectare.drop(columns='coords'), coord_df], axis=1)

print(hectare_final)


In [ ]:
squirrel_icon_url = "squirrel.png"  # Cute squirrel icon

squirrel_layer = folium.FeatureGroup(name="Squirrels")

for index, row in nyc_squirrels.iterrows():
    icon = folium.CustomIcon(
        icon_image=squirrel_icon_url,
        icon_size=(20, 20)  # Adjust icon size as needed
    )
    folium.Marker(
        location=[row['Y'], row['X']],
        icon=icon,
        popup=row['Unique Squirrel ID']
    ).add_to(squirrel_layer)



In [ ]:
polygon_layer = folium.FeatureGroup(name="Hectare Polygons")

for index, row in hectare_final.iterrows():
    try:
        locations = [
            (row["lat_1"], row["lon_1"]),
            (row["lat_2"], row["lon_2"]),
            (row["lat_3"], row["lon_3"]),
            (row["lat_4"], row["lon_4"]),
            (row["lat_1"], row["lon_1"])
        ]
        folium.Polygon(
            locations=locations,
            color="red" if index % 2 == 0 else "blue",
            fill=True,
            fill_color="red" if index % 2 == 0 else "blue",
            fill_opacity=0.3,
            popup=str(row.get("id", f"Hectare {index}"))
        ).add_to(polygon_layer)
    except Exception as e:
        print(f"Error in polygon {index}: {e}")


In [ ]:
# Initialize the map
my_map_mc = folium.Map(location=[nyc_squirrels['Y'].mean(), nyc_squirrels['X'].mean()], zoom_start=15)

# Add layers
squirrel_layer.add_to(my_map_mc)
polygon_layer.add_to(my_map_mc)

# Add layer control toggle
folium.LayerControl().add_to(my_map_mc)

# Save or display
my_map_mc.save("interactive_squirrel_map.html")

 

In [ ]:
fur_colors = {
    "Gray": {
        "icon_url": "graysquirrel.png",  # Gray squirrel
        "layer": folium.FeatureGroup(name="Gray Squirrels")
    },
    "Cinnamon": {
        "icon_url": "squirrel.png",  # Cinnamon squirrel
        "layer": folium.FeatureGroup(name="Cinnamon Squirrels")
    },
    "Black": {
        "icon_url": "blacksquirrel.png",  # Black squirrel
        "layer": folium.FeatureGroup(name="Black Squirrels")
    }
}

for index, row in nyc_squirrels.iterrows():
    fur_color = row.get('Primary Fur Color', 'Unknown')

    if fur_color in fur_colors:
        icon = folium.CustomIcon(
            icon_image=fur_colors[fur_color]["icon_url"],
            icon_size=(20, 20)
        )
        folium.Marker(
            location=[row['Y'], row['X']],
            icon=icon,
            popup=row['Unique Squirrel ID']
        ).add_to(fur_colors[fur_color]["layer"])

polygon_layer = folium.FeatureGroup(name="Hectare Polygons")

for index, row in hectare_final.iterrows():
    try:
        locations = [
            (row["lat_1"], row["lon_1"]),
            (row["lat_2"], row["lon_2"]),
            (row["lat_3"], row["lon_3"]),
            (row["lat_4"], row["lon_4"]),
            (row["lat_1"], row["lon_1"])
        ]
        folium.Polygon(
            locations=locations,
            color="red" if index % 2 == 0 else "blue",
            fill=True,
            fill_color="red" if index % 2 == 0 else "blue",
            fill_opacity=0.3,
            popup=str(row.get("id", f"Hectare {index}"))
        ).add_to(polygon_layer)
    except Exception as e:
        print(f"Error in polygon {index}: {e}")

my_map_mc = folium.Map(location=[nyc_squirrels['Y'].mean(), nyc_squirrels['X'].mean()], zoom_start=15)


polygon_layer.add_to(my_map_mc)

for info in fur_colors.values():
    info["layer"].add_to(my_map_mc)

# Add toggle control
folium.LayerControl(collapsed=False).add_to(my_map_mc)

# Save and/or display
my_map_mc.save("grouped_squirrels_map.html")


In [ ]:
drop_columns_ap = ['Color notes', 'Specific Location', 'Other Activities', 'Other Interactions', 'Highlight Fur Color']
clean_squirrels_ap = nyc_squirrels.drop(columns = drop_columns_ap)
fear_columns_ap = ['Quaas', 'Moans', 'Tail flags', 'Runs from']
fear_ap = clean_squirrels_ap[fear_columns_ap].apply(pd.Series.value_counts)

clean_squirrels_ap["showed_fear_ap"] = clean_squirrels_ap[fear_columns_ap].applymap(
    lambda x: str(x).strip().lower() in ["yes", "true", "1"]).any(axis=1)


print(clean_squirrels_ap["showed_fear_ap"].value_counts())



In [ ]:
#fear per hectare
#saving the location of one squirrel identified in each hectare
squirrel_hec_id = nyc_squirrels[['X','Y','Hectare']]
squirrel_hec_id = squirrel_hec_id.sort_values(by = 'Hectare')
squirrel_hec_id.drop_duplicates(subset=["Hectare"], keep="first", inplace=True)
squirrel_hec_id.reset_index(drop=True, inplace=True)

#making and naming the polygons with shapely
hectares_shapely = []
for i in range(len(hectare_final)):
    p = (Polygon([(hectare_final.iloc[i,7],hectare_final.iloc[i,6]),
                  (hectare_final.iloc[i,9],hectare_final.iloc[i,8]),
                  (hectare_final.iloc[i,11],hectare_final.iloc[i,10]),
                  (hectare_final.iloc[i,13],hectare_final.iloc[i,12])]),
                 f'{hectare_final.iloc[i,1]}')
    hectares_shapely.append(p)

#finding which hectare each squirrel point belongs in
hectare_assign = []
for i in range(len(squirrel_hec_id)):
    p = Point(squirrel_hec_id.iloc[i,1],squirrel_hec_id.iloc[i,0])
    for s in hectares_shapely:
        if s[0].contains(p) == True:
            h = (([squirrel_hec_id.iloc[i,2],s[1]]))
            hectare_assign.append(h)
     
hectare_assign = pd.DataFrame(hectare_assign, columns = ['Hectare', 'id'])
hectare_assign['id'] = hectare_assign['id'].astype('int64')

#joinging the squirrels data with the hectare id number
nyc_squirrels = clean_squirrels_ap.merge(hectare_assign, how='left', on='Hectare')
nyc_squirrels

In [ ]:
# Group squirrels by hectare_id and calculate mean of 'showed_fear_ap'
fear_rates = nyc_squirrels.groupby("id")["showed_fear_ap"].mean().reset_index()
fear_rates.rename(columns={"showed_fear_ap": "fear_rate"}, inplace=True)

df_final = hectare_final.merge(fear_rates, on="id", how="left")

df_final["fear_label"] = df_final["fear_rate"].apply(
    lambda x: "Fear" if pd.notna(x) and x >= 0.5 else "No Fear"
)

In [ ]:
# # New FeatureGroup for labeled hectare polygons
# fear_polygons_layer = folium.FeatureGroup(name="Hectare Fear Zones")

# for index, row in df_final.iterrows():
#     try:
#         locations = [
#             (row["lat_1"], row["lon_1"]),
#             (row["lat_2"], row["lon_2"]),
#             (row["lat_3"], row["lon_3"]),
#             (row["lat_4"], row["lon_4"]),
#             (row["lat_1"], row["lon_1"])  # Close the polygon
#         ]

#         # Choose color based on fear_label
#         color = "red" if row["fear_label"] == "Fear" else "green"

#         folium.Polygon(
#             locations=locations,
#             color=color,
#             fill=True,
#             fill_color=color,
#             fill_opacity=0.4,
#             popup=f"Hectare ID: {row.get('hectare_id', index)}\nFear Level: {row['fear_label']}"
#         ).add_to(fear_polygons_layer)

#     except Exception as e:
#         print(f"Polygon error at index {index}: {e}")

# fear_polygons_layer.add_to(my_map_mc)

# folium.LayerControl(collapsed=False).add_to(my_map_mc)

# my_map_mc.save("map_with_fear_polygons.html")
# my_map_mc

In [ ]:
# Rebuild the map from scratch to avoid duplicates or conflicts
my_map_mc = folium.Map(location=[nyc_squirrels['Y'].mean(), nyc_squirrels['X'].mean()], zoom_start=15)

# New FeatureGroup for labeled hectare polygons
fear_polygons_layer = folium.FeatureGroup(name="Hectare Fear Zones")

for index, row in df_final.iterrows():
    try:
        locations = [
            (row["lat_1"], row["lon_1"]),
            (row["lat_2"], row["lon_2"]),
            (row["lat_3"], row["lon_3"]),
            (row["lat_4"], row["lon_4"]),
            (row["lat_1"], row["lon_1"])  # Close the polygon
        ]

        # Choose color based on fear_label
        color = "red" if row["fear_label"] == "Fear" else "green"

        folium.Polygon(
            locations=locations,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.4,
            popup=f"Hectare ID: {row.get('hectare_id', index)}\nFear Level: {row['fear_label']}"
        ).add_to(fear_polygons_layer)

    except Exception as e:
        print(f"Polygon error at index {index}: {e}")

fear_polygons_layer.add_to(my_map_mc)


# fear-based icons
showed_fear_mc = {
    "Fear": {
        "icon_url": "fear.png",
        "layer": folium.FeatureGroup(name="Fear")
    },
    "No Fear": {
        "icon_url": "happiness.png",
        "layer": folium.FeatureGroup(name="No Fear")
    }
}

for index, row in clean_squirrels_ap.iterrows():
    fear = row.get('showed_fear_ap', False)
    fear_label = "Fear" if fear else "No Fear"

    icon = folium.CustomIcon(
        icon_image=showed_fear_mc[fear_label]["icon_url"],
        icon_size=(20, 20)
    )
    folium.Marker(
        location=[row['Y'], row['X']],
        icon=icon,
        popup=row['Unique Squirrel ID']
    ).add_to(showed_fear_mc[fear_label]["layer"])

# Add fear layers to map
for info in showed_fear_mc.values():
    info["layer"].add_to(my_map_mc)

# Add the layer control toggle
folium.LayerControl(collapsed=False).add_to(my_map_mc)

# Save or display
my_map_mc.save("fear_map.html")

